In [7]:
%cd ..
%pwd
from demo import *

C:\Users\IRandman\Documents\devit


In [9]:
def main(
        # config_file="../configs/open-vocabulary/lvis/vitl.yaml",
        config_file="configs/few-shot/vitl_shot5.yaml",
        # rpn_config_file="../configs/RPN/mask_rcnn_R_50_FPN_1x.yaml",
        rpn_config_file="configs/RPN/mask_rcnn_R_50_C4_1x_fewshot_14.yaml",
        # model_path="../weights/trained/open-vocabulary/lvis/vitl_0069999.pth",
        model_path="weights/trained/few-shot/vitl_0089999.pth",

        image_dir='demo/input/input_boat',
        output_dir='demo/output',
        category_space="demo/boat_prototypes.pth",
        device='cuda:0',
        overlapping_mode=True,
        topk=1,
        output_pth=False,
        threshold=0.45
    ):
    assert osp.abspath(image_dir) != osp.abspath(output_dir)
    os.makedirs(output_dir, exist_ok=True)

    config = get_cfg()
    config.merge_from_file(config_file)
    config.DE.OFFLINE_RPN_CONFIG = rpn_config_file
    config.DE.TOPK = topk
    config.MODEL.MASK_ON = False

    config.freeze()
    
    augs = utils.build_augmentation(config, False)
    augmentations = T.AugmentationList(augs) 

    # building models
    model = Trainer.build_model(config).to(device)
    model.load_state_dict(torch.load(model_path, map_location=device)['model'])
    model.eval()
    model = model.to(device)

    if category_space is not None:
        category_space = torch.load(category_space)

        if len(category_space["label_names"]) < 2:
            category_space["label_names"].append("_blank_")
            real_prototypes = category_space["prototypes"]
            blank_prototypes = torch.zeros(1, 1024)
            category_space["prototypes"] = torch.cat((real_prototypes, blank_prototypes), 0)
        model.label_names = category_space["label_names"]
        model.test_class_weight = category_space["prototypes"].to(device)
        
    label_names =  model.label_names
    if 'mini soccer' in label_names: # for YCB
        label_names = list_replace(label_names, old='mini soccer', new='ball')

    for img_file in tqdm(glob(osp.join(image_dir, '*'))):
        base_filename = osp.splitext(osp.basename(img_file))[0]

        dataset_dict = {}
        image = utils.read_image(img_file, format="RGB")
        dataset_dict["height"], dataset_dict["width"] = image.shape[0], image.shape[1]

        aug_input = T.AugInput(image)
        augmentations(aug_input)
        dataset_dict["image"] = torch.as_tensor(np.ascontiguousarray(aug_input.image.transpose(2, 0, 1))).to(device)

        batched_inputs = [dataset_dict]

        output = model(batched_inputs)[0]
        output['label_names'] = model.label_names
        if output_pth:
            torch.save(output, osp.join(output_dir, base_filename + '.pth'))

        # visualize output
        instances = output['instances']
        boxes, pred_classes, scores = filter_boxes(instances, threshold=threshold)

        if overlapping_mode:
            # remove some highly overlapped predictions
            mask = box_area(boxes) >= 400
            boxes = boxes[mask]
            pred_classes = pred_classes[mask]
            scores = scores[mask]
            mask = ops.nms(boxes, scores, 0.3)
            boxes = boxes[mask]
            pred_classes = pred_classes[mask]
            scores = scores[mask]
            areas = box_area(boxes)
            indexes = list(range(len(pred_classes)))
            for c in torch.unique(pred_classes).tolist():
                box_id_indexes = (pred_classes == c).nonzero().flatten().tolist()
                for i in range(len(box_id_indexes)):
                    for j in range(i+1, len(box_id_indexes)):
                        bid1 = box_id_indexes[i]
                        bid2 = box_id_indexes[j]
                        arr1 = boxes[bid1].cpu().numpy()
                        arr2 = boxes[bid2].cpu().numpy()
                        a1 = np.prod(arr1[2:] - arr1[:2])
                        a2 = np.prod(arr2[2:] - arr2[:2])
                        top_left = np.maximum(arr1[:2], arr2[:2]) # [[x, y]]
                        bottom_right = np.minimum(arr1[2:], arr2[2:]) # [[x, y]]
                        wh = bottom_right - top_left
                        ia = wh[0].clip(0) * wh[1].clip(0)
                        if ia >= 0.9 * min(a1, a2): # same class overlapping case, and larger one is much larger than small
                            if a1 >= a2:
                                if bid2 in indexes:
                                    indexes.remove(bid2)
                            else:
                                if bid1 in indexes:
                                    indexes.remove(bid1)

            boxes = boxes[indexes]
            pred_classes = pred_classes[indexes]
        colors = assign_colors(pred_classes, label_names, seed=4)
        output = to_pil_image(draw_bounding_boxes(torch.as_tensor(image).permute(2, 0, 1), boxes, labels=[f'{label_names[cid]}_{score:.2f}' for cid, score in zip(pred_classes.tolist(), scores.tolist())], colors=colors))
        output.save(osp.join(output_dir, base_filename + '.out.jpg'))

In [10]:
main()

  0%|          | 0/2 [00:00<?, ?it/s]C:\Users\IRandman\.conda\envs\devit\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3191.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
C:\Users\IRandman\AppData\Local\Temp\ipykernel_26652\2027459221.py:112: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_numpy.cpp:205.)
  output = to_pil_image(draw